In [1]:
%matplotlib inline
from IPython.core.debugger import set_trace
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set(style="darkgrid")
import numpy as np
import sys
import os
import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit import rdBase
from rdkit import DataStructs



In [2]:
from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'
    
#identifiers  = ['3-Methylheptane', 'Aspirin', 'Diethylsulfate', 'Diethyl sulfate', '50-78-2', 'Adamant']
#total_chem_name =['Diethyl sulfate']
#identifiers  = total_chem_name
#for ids in identifiers :
#    print(ids,',',CIRconvert(ids))
#print (sys.executable)
#print (sys.path)

In [3]:
def checkmolecule(target_smiles,smiles_L):
       a_smile=target_smiles
       start_mol = Chem.MolFromSmiles(a_smile)
       if start_mol is not None:
          for atom in start_mol.GetAtoms():
              if atom.GetSymbol() == '*':
                 atom.SetFormalCharge(0)
          a_smile_new=Chem.MolToSmiles(start_mol,isomericSmiles=False)
#          print ("a_smile_new",a_smile_new)
       else:
          print ("wrong input of target smiles and stop")
          sys.exit() 
       Flag_match=False
       for i in range(len(smiles_L)):
           b_smile=smiles_L[i]
           removed_mol= Chem.MolFromSmiles(b_smile)
           if removed_mol is not None:
              for atom in removed_mol.GetAtoms():
                  if atom.GetSymbol() == '*':
                     atom.SetFormalCharge(0)
              b_smile_new=Chem.MolToSmiles(removed_mol,isomericSmiles=False)
              if a_smile_new==b_smile_new:
                 Flag_match=True
                 break
           #   truncate = Chem.DeleteSubstructs(start_mol,removed_mol, \
           #        onlyFrags=True,useChirality=False)
           #   if (truncate.GetNumAtoms()==0) and (a_smile_new==b_smile_new):
       return Flag_match

#smiles_L=['have not found','CCC','COC','O=C=O']
#target_smiles='O=C=O'
#if checkmolecule(target_smiles,smiles_L):
#   print("good test")
#   sys.exit()
#else:
#    print("not found")
#    sys.exit()
    

In [4]:
def checksmarts(target_smarts,smiles_L):
       Flag_match=False
       for i in range(len(smiles_L)):
           b_smile=smiles_L[i]
           main_smiles= Chem.MolFromSmiles(b_smile)
           if main_smiles is not None:
              if main_smiles.HasSubstructMatch(target_smarts):
                 Flag_match=True
                 break
       return Flag_match

#smiles_L=['have not found','CCC','COC','CC']
#target_smarts=Chem.MolFromSmarts('[#7X3;H2,H1,H0]')
#if checksmarts(target_smarts,smiles_L):
#    print("good test")
#    sys.exit()
#else:
#    print("not found")
#    sys.exit()

In [6]:
import re
import csv
from chemdataextractor import Document
from chemdataextractor.reader import PdfReader,AcsHtmlReader,RscHtmlReader,NlmXmlReader,UsptoXmlReader,CsspHtmlReader,XmlReader,HtmlReader,PlainTextReader
import cirpy
from pdfminer.pdfdocument import PDFDocument
file =open('test.html', 'rb')
doc=Document.from_file(file)
# the below test to read the text
csvfile="abstractwebofscience.csv"
fp=open(csvfile,"r")
csv_reader = csv.reader(fp, delimiter=',')
if '.' in csvfile:
   chaposition=csvfile.index('.')
else:
   chaposition=len(csvfile)-1
substringfilename=csvfile[:chaposition]
csvfilenew=substringfilename+'_new.csv'
csv_data = open(csvfilenew, 'w')
csvwriter = csv.writer(csv_data)
col_abstract=-1
for ix, row in enumerate(csv_reader):
    if ix ==0:
       for j in range(len(row)):
           if row[j].lower()=='abstract':
                col_abstract=j
                break
       if col_abstract==-1:
          print("csv file does not contain 'abstract' keyword and stop")
          sys.exit()
       csv_head=row
       csv_head.append('Interestring or not')
       csvwriter.writerow(csv_head)
    else:
       row_csv=row
       text_string=row[col_abstract]
       doc=Document(text_string)
       total_chem_name=[]
       for i in range(len(doc.cems)):
          total_chem_name.append(doc.cems[i].text)
       identifiers  = total_chem_name
       smiles_L=[]
       for ids in identifiers :
         #print(ids,',',CIRconvert(ids))
          smiles_L.append(CIRconvert(ids)) 
       if (len(smiles_L)!=len(identifiers)):
          print("something must be wrong because len(smiles_L)!=len(identifiers) and stop")
          sys.exit()     
#below check CO2 whether is inlcued in the abstract
       target_smiles='O=C=O'
       flag_target_smiles=False
       if checkmolecule(target_smiles,smiles_L):
          flag_target_smiles=True
# below check whether primary, second, or ternary amine are included in the abstract
       target_smarts=Chem.MolFromSmarts('[#7X3;H2,H1,H0]')
       flag_target_smarts=False
       if checksmarts(target_smarts,smiles_L):
          flag_target_smarts=True
          #for i in range(len(smiles_L)):
           # print (identifiers[i],smiles_L[i])
          #sys.exit()
       if flag_target_smiles and flag_target_smarts:
          row_csv.append('yes')
       else:
          row_csv.append('no')  
       csvwriter.writerow(row_csv)                 
fp.close()
csv_data.close()
       
#set_trace()

#text_string='Protonation and carbamate formation are critical reactions for CO2 capture using amine-based absorbents which impact both CO2 absorption and amine regeneration. In this regard, thermodynamic analysis towards CO2 ab-sorption in N-methylaminoethanol solution is carried out with specific attention to the corresponding protonation and carbamate formation reactions. The CO2 absorption mechanism is investigated with the acquisition of reaction equilibrium constant and heat of reaction. CO2 absorption performance of N-methylaminoethanol solution was evaluated in terms of CO2 loading and solution pH. A thermodynamic model was then developed to mathematically describe the investigated system and make prediction on equilibrium CO2 solubility, species profiles, and absorption/regeneration heat. Results show that the average relative deviation of equilibrium CO2 solubility calculated from the is 4.2%. The predictive species profile and CO2 absorption heat of N-methylaminoethanol solution indicates less energy cost to desorb CO2. This is in line with the relative unstable N-methylaminoethanol carbamate formation and consequential conversion to (bi)carbonate. The position of Nmethylaminoethanol as a potential absorbent in amine based carbon capture is shown in terms of chemical reaction constants, equilibrium CO2 solubility, second order rate constant (k(2)), and pKa.'
#doc=Document(text_string)
#print ("doc.elements=",len(doc.elements))
#print ("# of chemical names ",len(doc.cems))
#doc.cems
#total_chem_name=[]
#for i in range(len(doc.cems)):
#    total_chem_name.append(doc.cems[i].text)
#print (total_chem_name)
#doc = Document('UV-vis spectrum of 5,10,15,20-Tetra(4-carboxyphenyl)porphyrin in Tetrahydrofuran (THF).')
#doc.cems

